# Ch `11`: Concept `01`

## Multi RNN

All we need is TensorFlow:

In [1]:
import tensorflow as tf

First, define the constants. 

Let's say we're dealing with 1-dimensional vectors, and a maximum sequence size of 3.

In [2]:
input_dim = 1
seq_size = 3

Next up, define the placeholder(s). 

We only need one for this simple example: the input placeholder.

In [3]:
input_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, seq_size, input_dim])

Now let's make a helper function to create LSTM cells

In [4]:
def make_cell(state_dim):
    return tf.contrib.rnn.LSTMCell(state_dim)

Call the function and extract the cell outputs.

In [5]:
with tf.variable_scope("first_cell") as scope:
    cell = make_cell(state_dim=10)
    outputs, states = tf.nn.dynamic_rnn(cell, input_placeholder, dtype=tf.float32)

You know what? We can just keep stacking cells on top of each other. In a new variable scope, you can pipe the output of the previous cell to the input of the new cell. Check it out:

In [6]:
with tf.variable_scope("second_cell") as scope:
    cell2 = make_cell(state_dim=10)
    outputs2, states2 = tf.nn.dynamic_rnn(cell2, outputs, dtype=tf.float32)

What if we wanted 5 layers of RNNs? 

There's a useful shortcut that the TensorFlow library supplies, called `MultiRNNCell`. Here's a helper function to use it:

In [7]:
def make_multi_cell(state_dim, num_layers):
    cells = [make_cell(state_dim) for _ in range(num_layers)]
    return tf.contrib.rnn.MultiRNNCell(cells)

Here's the helper function in action:

In [8]:
multi_cell = make_multi_cell(state_dim=10, num_layers=5)
outputs5, states5 = tf.nn.dynamic_rnn(multi_cell, input_placeholder, dtype=tf.float32)

Before starting a session, let's prepare some simple input to the network.

In [9]:
input_seq = [[1], [2], [3]]



Start the session, and initialize variables.

In [10]:
init_op = tf.global_variables_initializer()

sess = tf.InteractiveSession()
sess.run(init_op)

We can run the outputs to verify that the code is sound.

In [11]:
outputs_val, outputs2_val, outputs5_val = sess.run([outputs, outputs2, outputs5], 
                                                   feed_dict={input_placeholder: [input_seq]})
print(outputs_val)
print(outputs2_val)
print(outputs5_val)

[[[-0.0595012   0.01239615 -0.02821737  0.05782973  0.02280153
   -0.00853369 -0.03575901  0.03789692  0.07049078  0.00917231]
  [-0.18443729  0.01474649 -0.07368745  0.14000139  0.06114158
   -0.01630135 -0.08158955  0.10733734  0.20064893  0.04016431]
  [-0.35123432 -0.01229131 -0.12513027  0.21246517  0.10408669
   -0.02050561 -0.12086178  0.1876209   0.33600038  0.10451161]]]
[[[ 0.00078744  0.00271808  0.0006809  -0.00047728 -0.01430413
    0.00367989  0.00519382 -0.00270487  0.00318851 -0.00099343]
  [ 0.00486692  0.00895152 -0.00137144  0.00023559 -0.0481291
    0.01305772  0.01971616 -0.00829132  0.01299118 -0.00442055]
  [ 0.0153      0.01894589 -0.01083777  0.00376368 -0.09608416
    0.02765664  0.04316777 -0.01503063  0.02809603 -0.01359858]]]
[[[-1.2273813e-05 -6.2976842e-06 -8.3985988e-06  6.2900504e-06
    4.8350380e-06  1.6433044e-06  9.5611695e-06 -8.5426354e-06
   -5.5399514e-06  9.4979523e-06]
  [-7.2101131e-05 -2.9865219e-05 -5.7951776e-05  3.7555055e-05
    1.894908